In [3]:
from audio_diffusion_pytorch import AudioDiffusionModel
import torch
from tqdm import tqdm
from IPython.display import Audio
from pathlib import Path
import torchaudio
import torchaudio.transforms as T
import pytorch_lightning as pl
from torch.utils.data import random_split, DataLoader, Dataset
import torch.nn.functional as F
import wandb


In [4]:
torch.__version__

'1.13.0+cu117'

In [2]:
wandb.init(project="RemFX", entity="mattricesound")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mattricesound. Use `wandb login --relogin` to force relogin


In [3]:
SAMPLE_RATE = 22050
LENGTH = 2**17#round(5 * SAMPLE_RATE) 6 seconds

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
class GuitarDataset(Dataset):
    def __init__(self, root, length=LENGTH):
        self.files = list(Path().glob(f"{root}/**/*.wav"))
        self.resampler = T.Resample(48000, SAMPLE_RATE)
    
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, idx):
        x, sr = torchaudio.load(self.files[idx])
#         x = x.view() # Duplicate channel
        resampled_x = self.resampler(x)
        if resampled_x.shape[1] < LENGTH:
            resampled_x = F.pad(resampled_x, (0, LENGTH - resampled_x.shape[1]))
        elif resampled_x.shape[1] > LENGTH:
            resampled_x = resampled_x[:, :LENGTH]
        return resampled_x.to(device)

In [10]:
g = GuitarDataset(Path("Clean"))

In [11]:
x = g[10]
print(x.shape)

torch.Size([1, 131072])


In [12]:
data = DataLoader(GuitarDataset(Path("Clean")), batch_size=32)

In [13]:
dataiter = iter(data)
x = next(dataiter)

In [14]:
x[0].shape

torch.Size([1, 131072])

In [15]:
# wandb.log({"Audio": wandb.Audio(x[0].view(-1).numpy(), sample_rate=SAMPLE_RATE)})

In [7]:
model = AudioDiffusionModel(in_channels=1, 
                            patch_size=1,
                            multipliers=[1, 2, 4, 4, 4, 4, 4],
                            factors=[2, 2, 2, 2, 2, 2],
                            num_blocks=[2, 2, 2, 2, 2, 2],
                            attentions=[0, 0, 0, 0, 0, 0]
                           )
model = model.to(device)

In [9]:
fs = 22050
t = 2 ** 18 / 22050
samples = torch.arange(t * fs) / fs

for i in range(300, 8000):
    f = i
    signal1 = torch.sin(2 * torch.pi * f * samples)
    signal2 = torch.sin(2 * torch.pi * (f*2) * samples)
    stacked_signal = torch.stack((signal1, signal2)).unsqueeze(1)
    stacked_signal = stacked_signal.to(device)
    loss = model(stacked_signal)
    loss.backward() 
    if i % 10 == 0:
        print(i)



300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2210
2220
2230
2240
2250
2260
2270
2280
2290
2300
2310
2320
2330
2340
2350
2360
2370
2380
2390
2400
2410
2420
2430


In [10]:
# Sample 2 sources given start noise
noise = torch.randn(2, 1, 2 ** 18)
noise = noise.to(device)
sampled = model.sample(
    noise=noise,
    num_steps=10 # Suggested range: 2-50
) # [2, 1, 2 ** 18]

In [11]:
z = sampled[1]

In [12]:
z.shape

torch.Size([1, 262144])

In [13]:
Audio(z.cpu(), rate=22050)

In [14]:
Audio(sig[0], rate=22050)

NameError: name 'sig' is not defined

In [18]:
Audio(stacked_signal[1].cpu(), rate=22050)

In [47]:
i

7999

In [70]:
f = 4000
signal1 = torch.sin(2 * torch.pi * f * samples)
signal2 = torch.sin(2 * torch.pi * (f*2) * samples)

In [72]:
Audio(signal1.cpu(), rate=22050)

In [66]:
fs = 22050
f = 440
t = 2 ** 18 / 22050
samples = torch.arange(t * fs) / fs
signal = torch.sin(2 * torch.pi * f * samples)
sig = torch.stack((signal, signal)).unsqueeze(1)

In [7]:
sig.shape

torch.Size([2, 1, 262144])

In [17]:
262144 / 16

16384.0

In [89]:
signal.shape

torch.Size([264600])

In [17]:
Audio(sig[0], rate=22050)

In [24]:
epochs = 500
for i in tqdm(range(epochs)):
    for batch in data:
        loss = model(batch)
        loss.backward()
    if i % 10 == 0:
        wandb.log({"loss": loss})
        with torch.no_grad():
            noise = torch.randn(1, 1, 2**17).to(device)
            sampled = model.sample(noise=noise, num_steps=40)
            z = sampled.view(-1)
            wandb.log({f"Audio_{i}": wandb.Audio(z.cpu().numpy(), sample_rate=SAMPLE_RATE)})
            
            
        

100%|██████████| 500/500 [15:47:21<00:00, 113.68s/it]  


In [259]:
noise = torch.randn(1, 1, 2**17)
sampled = model.sample(noise=noise, num_steps=50)

In [260]:
print(sampled.shape, sampled)

torch.Size([1, 1, 131072]) tensor([[[-0.4879, -0.4534, -0.4094,  ..., -1.0000,  0.8554, -0.9605]]])


In [32]:
z = sampled.view(-1)
# z = z.mean(axis=0)